In [1]:
from helpers.examples.example_data import HOLE_METHOD_VARIATIONS
from helpers.visuals.visualize import Visualizer
from classes.workflow import Workflow
from classes.machine import Machine
from helpers.examples.example_gen import ExampleGen
from helpers.simulation.simulation import Simulation, save_sims_to_excel
from classes.scheduler import FillMethod, Scheduler
from helpers.pick_best_method import find_seed_for
import random
from itertools import product
import xlsxwriter
from algos.ruin_and_recreate import RuinRecreate
from helpers.utils import find_perc_diff


In [2]:
def write_to_excel(schedules, wks, bold, pos):

    if pos is None:
        pos = [2, 0]

    # The image we insert needs at least 20 rows.
    x_offset = pos[0]
    y_offset = pos[1] + 1
    # imgdata = io.BytesIO()
    # fig.savefig(imgdata, format='png')
    # wks.insert_image(s_pos[0], s_pos[1], '', {'image_data': imgdata})

    # The image we insert needs at least 20 rows.
    x_offset = 0 
    y_offset = 0

    # wks.set_column(0, 50, 20)

    # Headers
    headers = ["heft", "cpop", "hheft", "rr_random", "n", "fat", "density", "regularity", "jump"]
    for i, h in enumerate(headers):
        wks.write(x_offset, y_offset + i, h, bold)
         
    for s in schedules:
        x_offset += 1
        for i, h in enumerate(headers):
            wks.write(x_offset, y_offset + i, s.get(h), bold)

In [3]:
minalpha = 20
maxalpha = 50
n = [50, 100, 200, 300, 400, 500]
fat = [0.1, 0.2, 0.4, 0.6, 0.8]
density = [0.2, 0.8]
regularity = [0.2, 0.8]
jump = [1, 2, 4]
random.seed(0)
network = 500
ruin_methods = ["random"]

keys = ['n', 'fat', 'density', 'regularity', 'jump']
values = [n, fat, density, regularity, jump]
workbook = xlsxwriter.Workbook('all_machines.xlsx')

In [4]:
# random.seed(0)
# run_methods = [HOLE_METHOD_VARIATIONS['EFT_variations'], HOLE_METHOD_VARIATIONS['compositions'], HOLE_METHOD_VARIATIONS['criticals_unsorted'], HOLE_METHOD_VARIATIONS['holes-paper-2011']]
# run_methods = sum(run_methods, list())
# workbook = xlsxwriter.Workbook('multiple_wfs.xlsx')
# network = 500
# for n_machines in [4, 8, 16, 32]:
# # for n_machines in [4]:
#     bold = workbook.add_format({'bold': True})
#     wks = workbook.add_worksheet(f'Machines {n_machines}')
#     schedules = []
#     for n_workflows in [5, 10, 20, 30]:

#         machines = Machine.load_n_static_machines(n_machines, network * 125)
#         workflows = Workflow.load_random_workflows(machines, n_workflows)
#         print('Number of workflows: ', n_workflows)

#         # Sim
#         sim = Simulation(run_methods, machines, workflows)
#         sim.run()
#         for s in sim.schedulers:
#             print(s.method_used_info())
#             schedules.append({"method": s.method_used_info(concise=True), "s_len": s.schedule_len, "avg_util": s.machines_util_avg_perc, "avg_wf_len": s.avg_workflow_makespan})

#         print("----------------------------------")
#     print()
#     write_to_excel(schedules, wks, bold, [0, 0])

In [5]:
for n_machines in [2, 4, 8, 16, 32, 64]:
    bold = workbook.add_format({'bold': True})
    wks = workbook.add_worksheet(f'Machines {n_machines}')
    schedulers = []
    for v in product(*values):
        param = dict(zip(keys, v))
        filename = "./data/generated_dags/{}_{}_{}_{}_{}.dot".format(param['n'], param['fat'],
                                                                    param['density'],
                                                                    param['regularity'],
                                                                    param['jump'])
        if param['fat'] == 0.1 and param['jump'] == 1:
            print(filename, ' pipeline skipped')
            continue
        param = dict(zip(keys, v))
        machines = Machine.load_n_static_machines(n_machines, network * 125)
        wf = Workflow(id_=0, file_path=filename, wf_type="random", machines=machines, add_dummies=True)
        print(filename)
        # machines = Machine.load_n_static_machines(n_machines, network * 125)
        # wf = Workflow(id_=0, file_path="./data/generated_dags/100_0.8_0.2_0.8_2.dot", wf_type="random", machines=machines, add_dummies=True)
        # wf = Workflow(id_=0, file_path="./data/epigenomics/epigenomics_100.json", wf_type="epigenomics", machines=machines, add_dummies=True)
        # print(f"n-wfs: {n} machines: {n_machines}  network: {network}")

        # Heft
        Scheduler.heft(wf.tasks, wf.machines)
        wf.set_scheduled(is_scheduled=True)
        idle_perc_heft = sum([m.get_idle_perc() for m in machines]) / len(machines)
        heft_len = wf.wf_len
        print("Heft: ", int(wf.wf_len), end=' ')

        # CPOP
        wfs, machines = ExampleGen.re_create_example([wf], machines, reset_task_status=True)
        cwf = wfs[0]
        Scheduler.cpop(cwf)
        cwf.set_scheduled(is_scheduled=True)
        cpop_len = cwf.wf_len
        print("CPOP: ", int(cpop_len), end=' ')

        # HHeft
        wfs, machines = ExampleGen.re_create_example([wf], machines, reset_task_status=True)
        hwf = wfs[0]
        Scheduler.heft_with_holes(hwf.tasks, hwf.machines, fill_method=FillMethod.FASTEST_FIT)
        hwf.set_scheduled(is_scheduled=True)
        hheft_len = hwf.wf_len
        print("HHeft: ", int(hheft_len), end=' ')

        # for rm in ruin_methods:
        rr = RuinRecreate(wf, machines, ruin_method="random")
        rr_wf, machines = rr.run()
        print("RR: ", int(hheft_len), end=' ')
        print("\n----------------------------------")
        param.update({'hheft': int(hheft_len), 'cpop': int(cpop_len), 'heft': int(heft_len), 'rr_random': int(rr_wf.wf_len)})
        schedulers.append(param)
    print()
    write_to_excel(schedulers, wks, bold, [0, 0])

./data/generated_dags/50_0.1_0.2_0.2_1.dot  pipeline skipped
./data/generated_dags/50_0.1_0.2_0.2_2.dot
Heft:  323 CPOP:  369 HHeft:  292 RR:  292 
----------------------------------
./data/generated_dags/50_0.1_0.2_0.2_4.dot
Heft:  299 CPOP:  318 HHeft:  242 RR:  242 
----------------------------------
./data/generated_dags/50_0.1_0.2_0.8_1.dot  pipeline skipped
./data/generated_dags/50_0.1_0.2_0.8_2.dot
Heft:  326 CPOP:  329 HHeft:  282 RR:  282 
----------------------------------
./data/generated_dags/50_0.1_0.2_0.8_4.dot
Heft:  272 CPOP:  319 HHeft:  239 RR:  239 
----------------------------------
./data/generated_dags/50_0.1_0.8_0.2_1.dot  pipeline skipped
./data/generated_dags/50_0.1_0.8_0.2_2.dot
Heft:  303 CPOP:  345 HHeft:  291 RR:  291 
----------------------------------
./data/generated_dags/50_0.1_0.8_0.2_4.dot
Heft:  255 CPOP:  311 HHeft:  230 RR:  230 
----------------------------------
./data/generated_dags/50_0.1_0.8_0.8_1.dot  pipeline skipped
./data/generated_dags/50

In [6]:
workbook.close()